In [1]:
from mxnet import ndarray as nd
from mxnet import autograd as ag
from mxnet import gluon


In [2]:
N = 2
M = 100000

real_w = nd.array([2, -3.4])
real_b = nd.array([4.2]).asscalar()

X = nd.random_normal(shape=(M, N))

y = nd.add(nd.dot(X, real_w), real_b)
nd.broadcast_add(y, nd.random_normal(scale=0.1, shape=y.shape), out=y)


[ 4.6494617  4.301683  -3.490609  ...  3.4537337  1.0882893  4.6615725]
<NDArray 100000 @cpu(0)>

In [3]:
batch_size = 50000
dataset = gluon.data.ArrayDataset(X, y)
data_grabber = gluon.data.DataLoader(dataset, batch_size, shuffle=True, last_batch="rollover")

In [4]:
_iter = iter(data_grabber)

In [5]:
net = gluon.nn.Sequential()
net.add(gluon.nn.Dense(1))
gluon.nn.Dense

mxnet.gluon.nn.basic_layers.Dense

In [6]:
net.initialize()

In [7]:
loss_squre = gluon.loss.L2Loss()

In [10]:
trainer = gluon.Trainer(net.collect_params(),
                       'sgd',
                       {
                           'learning_rate': 0.1
                       })

In [ ]:
epochs = 5
for e in range(epochs):
    total_loss = 0
    for data, label in data_grabber:
        with ag.record():
            output = net(data)
            loss = loss_squre(output, label)
        loss.backward()
        trainer.step(batch_size)
        total_loss += nd.sum(loss).asscalar()
        print('Epoch %d, average loss: %f' % (e, loss.mean().asscalar()))
        

Epoch 0, average loss: 13.446604
Epoch 0, average loss: 10.918149
Epoch 1, average loss: 8.843031
Epoch 1, average loss: 7.111665
